In [1]:
# Importar librerías

import pandas as pd
import matplotlib as mlt
import numpy as np
import matplotlib.pyplot as plt
import cv2
import PyQt5

In [2]:
# Para que las gráficas se impriman no en el notebook, sino en una nueva pestaña
%matplotlib qt

# Crear la matrix y dar realidad física a la simulación

In [3]:
# Coordenadas de mundo
nm = 1e-9
um = 1e-6
mm = 1e-3

# Longitud de onda
w_length = 650*nm


# Solo por operación definimos N y M de una imagen 2048 x 2048
N = 2048 # coordenada X
M = 2048 # coordenada Y

# Diferenciales, como N y M pendientes por definir para una imagen

dx = dy = 1*um # los diferenciales (es el tamaño de pixel)


# Definamos algunas funciones de interés

In [4]:
# Función para graficar complejos
def fun_ploteo_complejo(mat, indicador, escala, mapa_color = 'gray'):
    '''
    mat  es una matriz compleja a plotear
    indicador = I para intensidad, A para amplitud, P para fase
    escala = 1 para nada, 0 para logarítmica
    '''
    if(indicador =="I"):
        mat = np.abs(mat)**2
    elif(indicador == "A"):
        mat = np.abs(mat)
    elif(indicador == "P"):
        mat = np.angle(mat)

    if (escala == 1):
        plt.figure()
        plt.imshow(mat, cmap = mapa_color)
        plt.colorbar()
        plt.show()
    else:    
        plt.figure()
        plt.imshow( np.log( mat + 0.0001), cmap = mapa_color)
        plt.colorbar()
        plt.show()

#----------------------------------------------------------------------------------------------------------------
# Función que convierte campos de 3 canales a un solo canal 
def fun_monocromador(campo_colorado, canal = 0):
    '''
    fun_monocromador es una función que extrae los datos de un solo canal de un campo, se puede entender
    como si un campo se compone de diferentes frecuencias, seleccionar solo los valores de esa frecuencia en 
    en particular que conforman el campo. 
    
    Esta función acepta solo campos con más de un solo canal, si mete de 1 canal, saldrán datos incorrectos

    Variables de entrada
     - campo_colorado: (numpy.ndarray) ; es el campo al cual se filtrará solo los componentes de una frecuencia que
                                         lo consituyen, tiene dimensiones L_x,L_y,canales
     - canal: (int) ; es el índice de los datos correspondientes a la frecuencia que se desea seleccionar  
                      por ejemplo una imagen RGB tiene las frecuencias de rojo en canal = 0, verde en canal = 1,
                      y azul en canal = 2.
                      Por defecto se escoge el canal 0                                 
    '''

    campo_monocromático = campo_colorado[:,:,canal] #Tomar todas las filas y columnas, pero solo los valores del canal
                                                    # de interés

    return (campo_monocromático)

#----------------------------------------------------------------------------------------------------------------
# Función para "Pading"
def fun_pad(campo):
    '''
    fun_pad es una función que "padea" un campo (imagen) 2D con ceros tal que duplica su longitud y altura

    Solo recibe imágenes de 1 canal

    Variables de entrada:
     - campo: (numpy.ndarray) ; es el campo a padear de L_x y L_y dimensiones físicas reales

    Variables de salida:
     - campo_paded: (numpy.ndarray) ; es el campo padeado con ceros de dimensiones 2*L_x y 2*L_y 
    '''
    # extraer las dimensiones del pad
    dimension_pad = np.array( [np.shape(campo)[0] , np.shape(campo)[1] ] ) / 2

    # padear el campo con ceros
    campo_paded = np.pad(campo, ( (int(dimension_pad[0]), int(dimension_pad[0]) ), (int(dimension_pad[1]), int(dimension_pad[1])) ), mode = 'constant', constant_values = (0,0) )

    return (campo_paded)

#----------------------------------------------------------------------------------------------------------------
# Función extractora de características de un campo (imagen) 2D

# Nota: Función por depurar y mejorar,, no está terminada, saltarla
def fun_extractor(campo, L_x, L_y, dx, dy):
    '''
    fun_extractor es una función que extrae las características de interés de un campo (imagen) 2D
    que puede ser de 1 o  3 canales de tamaño con dimensiones físicas reales L_x * L_y que fue medida
    con pixeles de tamaño dx,dy 

    Variables de entrada:
    - campo: (numpy.ndarray) ; Es la imagen que vamos a difractar
    - L_x: (int) ; Dimension x del campo
    - L_y: (int) ; Dimension y del campo
    - dx: (int) ; Dimension x del pixel
    - dy: (int) ; Dimension y del pixel

    Variables de salida:
    - campo_data: (numpy.ndarray) ; Arreglo 2x2, su primer elemento tiene el número de muestras en x y en y
                                  el segundo tiene los deltas en x y en y.

    '''
    # Tamaño de la muestra
    N_x = int(L_x/dx)
    N_y = int(L_y/dy)


    campo_data =np.array( [N_x, N_y] )

    return(campo_data)

#----------------------------------------------------------------------------------------------------------------
# Función del kernel de una transformada discreta de Fourier y su inversa exp(+-(1j/(2pi))*(n*p/N_x + m*q*N_y))

def fun_fourker( cont_n_m, cont_p_q, N_array, ift = False ):
    '''
    Función  fun_fourker: es una función que calcula la exponencial exp(+-(1j/(2pi*N))*(p*n + q*m)) siendo el signo
    + para la transformada invera y - para la transformada normal
    
    Nota: 1j lo utilizamos para representar el complejo i

    Variables de entrada:
     - cont_n_m: (np.ndarray 1x2) ; arreglo que contiene los contadores n y m del espacio real (x,y)
     - cont_p_q: (np.ndarray 1x2) ; arreglo que contiene los contadores p y q del espacio recíprocro (fx,fy)
     - N_array: (np.ndarray 1x2) ; arreglo que contiene el número de las muestras N_x en x y N_y en y
     - ift: (bool) ; indica si se está calculando transformada o su inversa, por defecto es False
    
    Variables de salida:
     - fourker: (complex) ; valor exponencial evaluada con las variables de entrada

    '''
    #descomprimir
    n = cont_n_m[0]
    m = cont_n_m[1]

    p = cont_p_q[0]
    q = cont_p_q[1]

    N_x = N_array[0]
    N_y = N_array[1]

    
    if(ift == False):
        fourker = np.exp(-(2j*np.pi * (n*p/N_x + m*q/N_y) ) )
    else:
        fourker = np.exp( (2j*np.pi * (n*p/N_x + m*q/N_y) ) )
    return fourker
    
#----------------------------------------------------------------------------------------------------------------
# Función que calcua la transformada discreta de un campo ya sea real o reciproco

def fun_DFT(campo, L_x, L_y, dx, dy, ift = False):
    '''
    fun_DFT es una función que calcula la Transformada de Fourier Discreta (DFT) o inversa (iDFT) de un campo sin 
    tener en cuenta los deltas 

    Variables de entrada:
     - campo: (numpy.ndarray) ; campo (arreglo matricial) al cual campo de que se calculará la DFT o iDFT
     - L_x: (int) ; dimensión real del campo en el eje x
     - L_y: (int) ; dimensión real del campo en el eje y
     - dx: (float) ; espaciamento entre muestra y muestra en el eje x, se puede entender como la longitud del pixel
     - dy: (float) ; espaciamento entre muestra y muestra en el eje y, se puede entender como la altura del pixel
     - ift: (bool) ; Si True, calcula la iDFT, si False calcula DFT

    Variables de salida:
     - campo_transformado: (numpy.ndarray) ; matriz de igual dimensiones que campo que contiene los valores de 
                                             la DFT o iDFT 
    '''

    # Crear arreglo lleno de ceros de mismas dimensiones que campo donde guardaré la información de la transformada
    campo_transformado = np.zeros(np.shape(campo), dtype = complex)



    # Dimensiones de campo para contadores
    N_x = int(L_x/dx)
    N_y = int(L_y/dy)

    # definir una variable que alberge el valor de las sumas
    suma = 0

    # variable de escalada si se hace DFT o iDFT
    if(ift == False):
        escalador = dx*dy
    else:
        escalador = 1/(L_x*L_y)

    # DFT
    for p in np.arange(N_x):
        for q in np.arange(N_y):

            for n in np.arange(N_x):
                for m in np.arange(N_y):
                    
                    suma += campo[n,m] * fun_fourker(n,m,p,q,N_x,N_y,ift)
 
            
            #Guardar el dato de la transformada para una frecuencia p,q
            campo_transformado[p,q] = suma

            # Reiniciar el sumador para calcular otra frecuencia p,q
            suma = 0
    
    # escalar el campo
    campo_transformado = campo_transformado * escalador

    return campo_transformado


def fun_DFT2():
    '''
    
    '''

    # Dimensiones del campo para los contadores
    N_x = int(L_x/dx)
    N_y = int(L_y/dy)

    # variable de escalada si se hace DFT o iDFT
    if(ift == False):
        escalador = dx*dy
    else:
        escalador = 1/(L_x*L_y)

    # Crear una matriz de N_x filas con N_y columnas donde cada elemento de la matriz es un arreglo 1x2 donde el primer
    # valor es el número de la fila y el segundo es el número de la columna

    # ejemplo siendo
    #  _                       _
    # | [0,0]    [0,1]    [0,2] |
    # | [1,0]    [1,1]    [1,2] |
    # | [2,0]    [2,1]    [2,2] |
    #  -                       -
    # la dimension es (N_x, N_y, 2)
    matriz_indexada = np.indices((N_x,N_y,1))
    matriz_indexada = np.concatenate((matriz_indexada[0], matriz_indexada[1]), axis=2)

    def funcionf(cont_p_q, campo):
        np.sum(map(lambda n,m: campo[n,m]*fun_fourker(),matriz_indexada))

    matriz_DFT = map

    







In [6]:
par = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [8]:
matpe = np.indices((5,5,1))
matpe = np.concatenate( (matpe[0], matpe[1]), axis=2)
print(matpe.shape)

[*map(lambda x: x+10, matpe ) ]

(5, 5, 2)


[array([[10, 10],
        [10, 11],
        [10, 12],
        [10, 13],
        [10, 14]]),
 array([[11, 10],
        [11, 11],
        [11, 12],
        [11, 13],
        [11, 14]]),
 array([[12, 10],
        [12, 11],
        [12, 12],
        [12, 13],
        [12, 14]]),
 array([[13, 10],
        [13, 11],
        [13, 12],
        [13, 13],
        [13, 14]]),
 array([[14, 10],
        [14, 11],
        [14, 12],
        [14, 13],
        [14, 14]])]

In [ ]:
index = np.indices((1000,1000,1))
index[0],index[1]

In [461]:
def fuff(x):
    x^2
    return x
[*map(lambda x: [*map(fuff, x)], np.array([[1,1],[2,2],[3,3]]) ) ]



[[1, 1], [2, 2], [3, 3]]

# Punto 1

Implemente numéricamente, en Python o MATLAB, los métodos de difracción escalar de Transformadad de Fresnel y Espectro Angular en sus formas de sumatorias discretas (DFT), y en la forma basada en transformadas rápidas de Fourier (FFT). Compare el desempeño de ambos métodos en términos del tiempo necesario para realizar el cómputo de un mismo experimento difractivo de su elección. ¿La mejora de desempe o coincide con sus expectativas?

In [329]:
war_img = cv2.imread('C:/Users/braya/Desktop/Git I Opticos/Instrumentos-Opticos/Entrega_01/c.jpeg')              # Lecutra del .csv como un objeto np.array
np.shape(war_img)
war_mono = fun_monocromador(war_img)
fun_ploteo_complejo(war_mono, "A" , 1)

### DFT de Espectro Angular

In [330]:
# Obtener el espectro del campo de entrada, Primera DFT

def fun_DFT_EspA(campo, w_length, z, L_x, L_y, dx, dy):
     '''
     
     '''
     # Vector de onda
     k_vect = 2*np.pi/w_length



     # Número de muestras
     N_x = int(L_x/dx)
     N_y = int(L_y/dy)

     # 1. Generar espectro del campo en 0
     A_spectra = fun_DFT(campo,L_x, L_y, dx, dy)

     # 2. Calcular el espectro a una distancia z

     # Matriz de ceros que albergará la información del espectro propagado a una distancia z
     A_spectra_z = np.zeros(np.shape(campo), dtype = complex)

     for p in np.arange(N_x):
          for q in np.arange(N_y):
               A_spectra_z[p,q] = A_spectra[p,q] * np.exp(z*1j*k_vect*np.sqrt(1-(w_length*1/L_x)**2 * (p**2 + q**2) ) )


     # 3. Calcular la DFT inversa para volver al dominio real
     campo_propagado = fun_DFT(campo, L_x, L_y, dx, dy, ift = True)

     # 4. Ordenar las frecuencias, dado que entramos a las frecuencias de la réplica, está desordenado.
     campo_propagado = np.fft.fftshift(campo_propagado)

     return(campo_propagado)


In [324]:
# fun_DFT_EspA(war_mono,1, 1, 1680, 2600, 1, 1)
fun_DFT(war_mono,1680,2600,1,1)

KeyboardInterrupt: 

In [310]:
np.shape(war_img)

(1680, 2600, 3)